In [1]:
import sqlite3
import pandas as pd

conn = sqlite3.connect('meal_schedule.db')
df = pd.read_sql("SELECT * FROM daily_meals", conn)
conn.close()

print(df.head(10))
print(f"Total meals: {len(df)}")


Empty DataFrame
Columns: [crew_name, day, meal, food_name, food_grams, food_rating, beverage_name, beverage_grams, beverage_rating]
Index: []
Total meals: 0


In [2]:
import sqlite3
import pandas as pd
import random

# Config
DB_PATH = 'beverage.db'
crew = ['Richard Pears', 'Harshita Anantula']
rating_range = (1, 5)  # inclusive

# Load beverage names
conn = sqlite3.connect(DB_PATH)
beverages = pd.read_sql("SELECT name FROM beverages", conn)['name'].tolist()

# Ensure ratings table exists
conn.execute("""
    CREATE TABLE IF NOT EXISTS beverage_ratings (
        crew_name TEXT,
        beverage_name TEXT,
        rating INTEGER,
        PRIMARY KEY (crew_name, beverage_name)
    );
""")

# Generate and insert ratings
for member in crew:
    for bev in beverages:
        rating = random.randint(*rating_range)
        conn.execute("""
            INSERT INTO beverage_ratings (crew_name, beverage_name, rating)
            VALUES (?, ?, ?)
            ON CONFLICT(crew_name, beverage_name) DO UPDATE SET rating=excluded.rating;
        """, (member, bev, rating))

conn.commit()
conn.close()

print(f"✅ Random beverage preferences generated for {crew} and saved to {DB_PATH}")


✅ Random beverage preferences generated for ['Richard Pears', 'Harshita Anantula'] and saved to beverage.db


In [15]:
import collections
import collections.abc
collections.Mapping = collections.abc.Mapping
from experta import *
import random



class MealSlot(Fact):
    """A meal that needs to be filled for a crew member"""
    pass

class FoodOption(Fact):
    """A candidate food with calories and rating"""
    pass

class BeverageOption(Fact):
    """A candidate beverage with calories and rating"""
    pass

class MealHistory(Fact):
    """Tracks the last food and beverage assigned"""
    pass

class MealAssigned(Fact):
    """Represents a filled meal slot"""
    pass

class SelectedFood(Fact):
    """Represents a selected food assignment for a slot"""
    pass

class SelectedBeverage(Fact):
    """Represents a selected beverage assignment for a slot"""
    pass


class MealPlanner(KnowledgeEngine):
    def __init__(self, name, calorie_target, food_list, beverage_list, start_day, food_ratings, beverage_ratings, duration, water_per_meal=250, ration_fraction=1.0):
        super().__init__()
        self.name = name
        self.start_day = start_day
        self.calorie_target = calorie_target
        self.duration = duration
        self.water_per_meal = water_per_meal
        self.meals_per_day = 3
        self.per_meal_kcal = calorie_target / self.meals_per_day
        self.schedule = []
        self.ration_fraction = ration_fraction
        self.prev_foods = set()
        self.prev_beverages = set()

        # ✅ Store the raw rating dictionaries
        self.food_ratings = {k.lower(): v for k, v in food_ratings.items()}
        self.beverage_ratings = {k.lower(): v for k, v in beverage_ratings.items()}

        self.foods = []
        for f in food_list:
            name = f.get('food_name') or f.get('name')
            if name:
                rating = self.food_ratings.get(name.lower())
                if rating is not None:
                    f['food_name'] = name
                    f['rating'] = rating
                    self.foods.append(f)

        self.beverages = []
        for b in beverage_list:
            name = b.get('beverage_name') or b.get('name')
            if name:
                rating = self.beverage_ratings.get(name.lower())
                if rating is not None:
                    b['beverage_name'] = name
                    b['rating'] = rating
                    self.beverages.append(b)

    def setup(self):
        self.reset()
        # Declare all slots for the mission
        for i in range(self.duration):
            day = self.start_day + i
            for meal in range(1, self.meals_per_day + 1):
                self.declare(MealSlot(day=day, meal=meal, crew_name=self.name))

        # Declare all food and beverage options
        for food in self.foods:
            self.declare(FoodOption(food_name=food['food_name'],
                                    calories_per_gram=food['calories_per_gram'],
                                    rating=food['rating']))

        for bev in self.beverages:
            self.declare(BeverageOption(beverage_name=bev['beverage_name'],
                                        calories_per_gram=bev['calories_per_gram'],
                                        rating=bev['rating']))

        self.meal_history_fact = self.declare(MealHistory(crew_name=self.name, last_food=None, last_bev=None))
        print(f"[{self.name}] Foods available: {len(self.foods)}")
        print(f"[{self.name}] Beverages available: {len(self.beverages)}")


    def slot_already_assigned(self, fact_type, day, meal):
        return any(
            isinstance(f, fact_type) and f['day'] == day and f['meal'] == meal
            for f in self.facts.values()
        )
    
    def weighted_choice(self, candidates, weight_key):
        total = sum(f[weight_key] for f in candidates)
        r = random.uniform(0, total)
        upto = 0
        for f in candidates:
            if upto + f[weight_key] >= r:
                return f
            upto += f[weight_key]
        return candidates[-1]


    @Rule(
        MealSlot(day=MATCH.day, meal=MATCH.meal, crew_name=MATCH.name),
        AS.history << MealHistory(crew_name=MATCH.name, last_food=MATCH.last_food, last_bev=MATCH.last_bev),
        AS.food << FoodOption(food_name=MATCH.fname, calories_per_gram=MATCH.cpg, rating=MATCH.rating),
        TEST(lambda fname, last_food: last_food is None or fname != last_food),
        salience=2
    )
    def assign_food(self, food, fname, cpg, rating, day, meal, name, last_food, last_bev):
        if self.slot_already_assigned(SelectedFood, day, meal):
            return

        # Prefer unrated items not recently used
        candidates = [f for f in self.foods if f['food_name'] != last_food]

        # Filter out recently used foods (except when pool is small)
        if len(candidates) > 5:
            candidates = [f for f in candidates if f['food_name'] not in self.prev_foods] or candidates

        # Pick one with weighted preference
        chosen = self.weighted_choice(candidates, 'rating')
        grams = round(self.per_meal_kcal / chosen['calories_per_gram'], 2)

        self.prev_foods.add(chosen['food_name'])
        self.declare(SelectedFood(
            food=chosen['food_name'],
            food_grams=grams,
            day=day,
            meal=meal
        ))
        pass



    @Rule(
        MealSlot(day=MATCH.day, meal=MATCH.meal, crew_name=MATCH.name),
        AS.history << MealHistory(crew_name=MATCH.name, last_food=MATCH.last_food, last_bev=MATCH.last_bev),
        AS.bev << BeverageOption(beverage_name=MATCH.bname, calories_per_gram=MATCH.cpb, rating=MATCH.rating),
        TEST(lambda bname, last_bev: last_bev is None or bname != last_bev),
        salience=2
    )
    def assign_beverage(self, bev, bname, cpb, rating, day, meal, name, last_food, last_bev):
        if self.slot_already_assigned(SelectedBeverage, day, meal):
            return

        candidates = [b for b in self.beverages if b['beverage_name'] != last_bev]

        # Reduce repeats if possible
        if len(candidates) > 3:
            candidates = [b for b in candidates if b['beverage_name'] not in self.prev_beverages] or candidates

        chosen = self.weighted_choice(candidates, 'rating')

        self.prev_beverages.add(chosen['beverage_name'])
        self.declare(SelectedBeverage(
            beverage=chosen['beverage_name'],
            beverage_grams=self.water_per_meal,
            day=day,
            meal=meal
        ))



    @Rule(
        AS.food_f << SelectedFood(day=MATCH.day, meal=MATCH.meal, food=MATCH.food, food_grams=MATCH.fg),
        AS.bev_f << SelectedBeverage(day=MATCH.day, meal=MATCH.meal, beverage=MATCH.bev, beverage_grams=MATCH.bg),
        AS.slot << MealSlot(day=MATCH.day, meal=MATCH.meal, crew_name=MATCH.name),
        AS.history << MealHistory(crew_name=MATCH.name, last_food=MATCH.last_food, last_bev=MATCH.last_bev),
        salience=1
    )
    def finalize_meal(self, food_f, bev_f, slot, day, meal, name, food, fg, bev, bg, history):
        print(f"✅ Finalizing Day {day}, Meal {meal} for {name}: {food} + {bev}")
        self.schedule.append({
            'day': day,
            'meal': meal,
            'food': food,
            'food_grams': fg,
            'beverage': bev,
            'beverage_grams': bg
        })
        self.modify(history, last_food=food, last_bev=bev)
        self.declare(MealAssigned(crew_name=name, day=day, meal=meal))
        self.retract(food_f)
        self.retract(bev_f)
        self.retract(slot)

    def plan_within_mass_budget(self, mass_budget, min_fraction=0.6, step=0.01):
        """
        Iteratively decrease ration_fraction until total_mass fits mass_budget,
        or until a complete schedule is generated.
        """
        fraction = 1.0
        expected_meals = self.duration * self.meals_per_day

        while fraction >= min_fraction:
            self.ration_fraction = fraction
            self.per_meal_kcal = (self.calorie_target * self.ration_fraction) / self.meals_per_day
            self.schedule.clear()
            result = self.run_planner()

            # ✅ Exit early if a full schedule was successfully generated
            if len(result['schedule']) >= expected_meals:
                result['ration_fraction'] = round(fraction, 3)
                return result

            # ✅ Also exit if total mass is already acceptable
            if result['total_mass'] <= mass_budget:
                result['ration_fraction'] = round(fraction, 3)
                return result

            fraction -= step

        # If all failed, return last attempted plan with a warning
        result['ration_fraction'] = round(fraction + step, 3)
        result['warning'] = f"Unable to meet mass budget of {mass_budget} kg above {min_fraction*100}% rationing."
        return result


    def run_planner(self):
        self.setup()
    
        self.run()  




        total_food_mass = round(sum(x['food_grams'] for x in self.schedule), 2)
        total_bev_mass = round(sum(x['beverage_grams'] for x in self.schedule), 2)
        total_intake_kcal = round(self.calorie_target * self.ration_fraction, 2)
        intake_ratio = total_intake_kcal / self.calorie_target

        # For future: adjust target downward and recalculate actual kcal intake to simulate rationing

        sufficiency = 'sufficient'
        intake_ratio = total_intake_kcal / self.calorie_target

        if intake_ratio < 0.85:
            sufficiency = 'insufficient'
        elif intake_ratio < 0.95:
            sufficiency = 'moderate'
        else:
            sufficiency = 'sufficient'

        return {
            'crew_member': self.name,
            'schedule': self.schedule,
            'total_food_mass': total_food_mass,
            'total_beverage_mass': total_bev_mass,
            'total_mass': round(total_food_mass + total_bev_mass, 2),
            'calorie_target': self.calorie_target,
            'rationed_kcal': total_intake_kcal,
            'intake_ratio': round(intake_ratio, 3),
            'sufficiency_status': sufficiency
        }


In [16]:
import sqlite3
import pandas as pd
# Config
crew_member = 'Richard Pears'
duration = 7
activity = 'moderate'
kcal_per_kg = {'low': 30, 'moderate': 40, 'daily': 50}
mass_budget = 10_000  # arbitrarily large to avoid triggering rationing

# 1. Load crew info
conn = sqlite3.connect('astronauts.db')
crew_df = pd.read_sql("SELECT * FROM crew", conn)
conn.close()

crew_mass = crew_df[crew_df['name'] == crew_member]['mass'].astype(float).values[0]
calorie_target = round(crew_mass * kcal_per_kg[activity], 2)

# 2. Load food and beverage data
food_df = pd.read_sql("SELECT * FROM foods", sqlite3.connect('nutrition.db'))
bev_df = pd.read_sql("SELECT * FROM beverages", sqlite3.connect('beverage.db'))

# 3. Load ratings
food_ratings_df = pd.read_sql("SELECT * FROM food_ratings", sqlite3.connect('nutrition.db'))
bev_ratings_df = pd.read_sql("SELECT * FROM beverage_ratings", sqlite3.connect('beverage.db'))

food_ratings = food_ratings_df[food_ratings_df['crew_name'] == crew_member].set_index('food_name')['rating'].to_dict()
bev_ratings = bev_ratings_df[bev_ratings_df['crew_name'] == crew_member].set_index('beverage_name')['rating'].to_dict()

# 4. Format data
# 4. Format data
food_list = food_df.rename(columns={'name': 'food_name'}).to_dict('records')
beverage_list = bev_df.rename(columns={'name': 'beverage_name'}).to_dict('records')

import pandas as pd
planner = MealPlanner(
    name=crew_member,
    calorie_target=calorie_target,
    food_list=food_list,
    beverage_list=beverage_list,
    food_ratings=food_ratings,
    beverage_ratings=bev_ratings,
    duration=duration,
    start_day=1
)

result = planner.plan_within_mass_budget(mass_budget)

print(f"✅ Planned {len(result['schedule'])} meals")
pd.DataFrame(result['schedule'])


# 7. Retry if any MealSlots are unassigned
if len(result['schedule']) < duration * planner.meals_per_day:
    print("🔁 Detected unfinalized meals")


# 8. Display output
pd.DataFrame(result['schedule'])


[Richard Pears] Foods available: 12
[Richard Pears] Beverages available: 12
✅ Finalizing Day 1, Meal 1 for Richard Pears: Beef Burrito + Iced Tea (sweetened)
✅ Finalizing Day 1, Meal 2 for Richard Pears: Chicken Caesar Salad + Sports Drink
✅ Finalizing Day 1, Meal 3 for Richard Pears: Chicken Alfredo Pasta + Energy Drink
✅ Finalizing Day 2, Meal 1 for Richard Pears: Beef Lasagna + Powdered Drink Mix (prepared)
✅ Finalizing Day 2, Meal 2 for Richard Pears: Beef Burrito + Powdered Drink Mix (prepared)
✅ Finalizing Day 2, Meal 3 for Richard Pears: Beef Lasagna + Fruit Punch
✅ Finalizing Day 3, Meal 1 for Richard Pears: Vegetable Stir-Fry + Energy Drink
✅ Finalizing Day 3, Meal 2 for Richard Pears: Turkey Sandwich + Fruit Punch
✅ Finalizing Day 3, Meal 3 for Richard Pears: Beef Burrito + Lemon-Lime Soda
✅ Finalizing Day 4, Meal 1 for Richard Pears: Baked Salmon + Orange Soda
✅ Finalizing Day 4, Meal 2 for Richard Pears: Cheese Omelette + Cola (diet)
✅ Finalizing Day 4, Meal 3 for Richard P

,day,meal,food,food_grams,beverage,beverage_grams
0,1,1,Beef Burrito,549.02,Iced Tea (sweetened),250
1,1,2,Chicken Caesar Salad,777.78,Sports Drink,250
2,1,3,Chicken Alfredo Pasta,583.33,Energy Drink,250
3,2,1,Beef Lasagna,652.68,Powdered Drink Mix (prepared),250
4,2,2,Beef Burrito,549.02,Powdered Drink Mix (prepared),250
5,2,3,Beef Lasagna,652.68,Fruit Punch,250
6,3,1,Vegetable Stir-Fry,1098.04,Energy Drink,250
7,3,2,Turkey Sandwich,373.33,Fruit Punch,250
8,3,3,Beef Burrito,549.02,Lemon-Lime Soda,250
9,4,1,Baked Salmon,448.72,Orange Soda,250


In [35]:
import sqlite3
import pandas as pd

crew_members = ['Alexis Lewis', 'Richard Pears', 'Harshita Anantula']

# Load foods and beverages
food_df = pd.read_sql("SELECT * FROM foods", sqlite3.connect('nutrition.db'))
bev_df = pd.read_sql("SELECT * FROM beverages", sqlite3.connect('beverage.db'))

print("✅ Foods table:")
display(food_df[['name', 'calories_per_gram']].head())

print("✅ Beverages table:")
display(bev_df[['name', 'calories_per_gram']].head())

for name in crew_members:
    print(f"\n=== 🍽️ {name} — Food Ratings ===")
    food_ratings_df = pd.read_sql("SELECT * FROM food_ratings WHERE crew_name = ?", sqlite3.connect('nutrition.db'), params=[name])
    display(food_ratings_df)

    print(f"\n=== 🥤 {name} — Beverage Ratings ===")
    bev_ratings_df = pd.read_sql("SELECT * FROM beverage_ratings WHERE crew_name = ?", sqlite3.connect('beverage.db'), params=[name])
    display(bev_ratings_df)

    # Normalize names
    rated_foods = set(food_ratings_df['food_name'].str.lower())
    available_foods = set(food_df['name'].str.lower())
    missing = rated_foods - available_foods
    print(f"❌ Unmatched rated food names for {name}: {missing}")

    rated_bevs = set(bev_ratings_df['beverage_name'].str.lower())
    available_bevs = set(bev_df['name'].str.lower())
    missing_bevs = rated_bevs - available_bevs
    print(f"❌ Unmatched rated beverage names for {name}: {missing_bevs}")


✅ Foods table:


,name,calories_per_gram
0,Grilled Chicken Breast,1.65
1,Spaghetti with Meat Sauce,1.33
2,Beef Lasagna,1.43
3,Vegetable Stir-Fry,0.85
4,Chicken Caesar Salad,1.20


✅ Beverages table:


,name,calories_per_gram
0,Cola (regular),0.47
1,Cola (diet),0.00
2,Orange Soda,0.54
3,Lemon-Lime Soda,0.48
4,Energy Drink,0.60



=== 🍽️ Alexis Lewis — Food Ratings ===


...

❌ Unmatched rated food names for Richard Pears: set()
❌ Unmatched rated beverage names for Richard Pears: set()

=== 🍽️ Harshita Anantula — Food Ratings ===


,crew_name,food_name,rating
0,Harshita Anantula,Baked Salmon,3
1,Harshita Anantula,Beef Burrito,3
2,Harshita Anantula,Beef Lasagna,3
3,Harshita Anantula,Cheese Omelette,2
4,Harshita Anantula,Chicken Alfredo Pasta,3
5,Harshita Anantula,Chicken Caesar Salad,2
6,Harshita Anantula,Grilled Chicken Breast,5
7,Harshita Anantula,Shrimp Fried Rice,2
8,Harshita Anantula,Spaghetti with Meat Sauce,2
9,Harshita Anantula,Tofu and Vegetable Curry,2



=== 🥤 Harshita Anantula — Beverage Ratings ===


,crew_name,beverage_name,rating
0,Harshita Anantula,Chocolate Milk (UHT),5
1,Harshita Anantula,Cola (diet),3
2,Harshita Anantula,Cola (regular),3
3,Harshita Anantula,Energy Drink,3
4,Harshita Anantula,Fruit Punch,5
5,Harshita Anantula,Iced Tea (sweetened),3
6,Harshita Anantula,Instant Coffee (prepared),4
7,Harshita Anantula,Lemon-Lime Soda,1
8,Harshita Anantula,Orange Soda,3
9,Harshita Anantula,Powdered Drink Mix (prepared),2


❌ Unmatched rated food names for Harshita Anantula: set()
❌ Unmatched rated beverage names for Harshita Anantula: set()


In [37]:
import sqlite3

def get_table_and_headers(db_path='gas_budget.db'):
    # Connect to the database
    conn = sqlite3.connect(db_path)
    cursor = conn.cursor()

    # Get all table names
    cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
    tables = cursor.fetchall()

    if not tables:
        print("No tables found in the database.")
        return

    for (table_name,) in tables:
        print(f"\n📋 Table: {table_name}")
        cursor.execute(f"PRAGMA table_info({table_name});")
        columns = cursor.fetchall()
        if columns:
            headers = [col[1] for col in columns]  # column name is in second position
            print("  ➤ Columns:", headers)
        else:
            print("  ⚠ No columns found.")

    conn.close()

get_table_and_headers()  # or pass a different path if needed



📋 Table: gas_masses
  ➤ Columns: ['id', 'timestamp', 'duration', 'crew_count', 'body_masses', 'activity', 'oxygen_tank_weight_per_kg', 'co2_generated', 'o2_required_kg', 'o2_reclaimed', 'o2_tank_mass', 'scrubber_mass', 'recycler_mass', 'total_gas_mass', 'use_scrubber', 'use_recycler', 'co2_scrubber_efficiency', 'scrubber_weight_per_kg', 'co2_recycler_efficiency', 'recycler_weight', 'within_limit', 'weight_limit']

📋 Table: sqlite_sequence
  ➤ Columns: ['name', 'seq']
